In [1]:

from PIL import Image
import numpy as np
import glob
import regex 
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

from sklearn.neighbors import KNeighborsClassifier


In [2]:



def read_dataset(path):
  data= np.zeros([400,10305])
  i=0
  images_list = []
  for filename in glob.glob(path+"/*/*.pgm"): 
    im=Image.open(filename)
    images_list.append(im)
    l=regex.split(r'/', filename)
    l2= regex.split(r's', l[1])
    arr = np.array(im)
    data[i,0:10304]=arr.reshape(1,10304)
    data[i, -1]=l2[1]
    i+=1
  return data, images_list

    

In [3]:
def data_splitting():
    test_X = np.zeros([200,10304])
    train_X = np.zeros([200,10304])
    test_y= np.zeros(200)
    train_y=np.zeros(200)
    for i in range(data.shape[0]):
        if i%2 == 0:
            test_X[int(i/2)]=data[i,:-1]
            test_y[int(i/2)]= data[i,-1]
        else:
            train_X[int((i-1)/2)]=data[i,:-1]
            train_y[int((i-1)/2)]= data[i,-1] 
    return train_X, train_y, test_X, test_y


In [4]:
def visualize(images,i):
  plt.imshow(images[i])
  

In [5]:
data, images= read_dataset("orl_faces") 
train_x, train_y, test_x, test_y= data_splitting()
#print(train_y)
#visualize(images, 32)

In [6]:
def FOTV(eig_vals,totVar,alpha):
    expVar=0.0
    i=0
    while expVar <= (alpha * totVar): 
        expVar+=eig_vals[i]
        i+=1
    return i

In [ ]:
def pCA(dataMatrix,alpha):
    X=dataMatrix[:,:]
    num_data, dim = X.shape
    mean_X = X.mean(axis=0)
    X = X - mean_X 
    df_mat = np.asmatrix(X)
    cov_mat = np.cov(df_mat.T)
    #cov_mat = np.dot(X, X.T)/(num_data) # covariance matrix
    eig_vals, eig_vecs = np.linalg.eig(cov_mat)
    totVar = sum(eig_vals)
    idx = eig_vals.argsort()[::-1]   
    eig_vals = eig_vals[idx]
    eig_vecs = eig_vecs[:,idx]
    num = FOTV(eig_vals,totVar,alpha)
    reducedVec=eig_vecs[:num,:]
    transformed = reducedVec.dot(df_mat)
    print(transformed.shape)
    plt.imshow(transformed[2].reshape((112,92)),cmap='gray')
    return transformed
pCA(train_x,0.9)

In [ ]:
def pca():
  pca = PCA(n_components= 200)
  pca_model= pca.fit(train_x)
  new_dimensions = pca_model.transform(train_x)  
  #plt.scatter(new_dimensions[:,0],new_dimensions[:,1])  
  #print(pca.explained_variance_ratio_)  
  #print(new_dimensions.shape)
  #print(pca.explained_variance_ratio_)
  return  new_dimensions

new_dimensions= pca()


In [ ]:
PCA(train_x)